In [1]:
from nuplan.database.nuplan_db_orm.nuplandb import NuPlanDB
from pathlib import Path
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper
from nuplan.database.nuplan_db_orm.frame import Frame
import os

from nuplan.planning.utils.multithreading.worker_pool import Task
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping

import os
import math
import argparse
import matplotlib.pyplot as plt
from tqdm import tqdm
from data_utils import *
from trajectory_tree_planner import *
from common_utils import get_filter_parameters, get_scenario_map
from data_process import DataProcessor
from train_utils import DrivingData
import numpy as np
from torch.utils.data import DataLoader

In [2]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # multi db files
data_root = "/cailiu2/Diffusion-Planner/data/"
map_path = "/cailiu2/Diffusion-Planner/data/maps"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"

In [3]:
path = ["/cailiu2/Diffusion-Planner/data/processed/us-nv-las-vegas-strip_7fc811bcf45f5e79.npz"] # the path must be list
train_set = DrivingData(path, 10, 10) # DrivingData继承自torch.Dataset，存储和加载npz数据
print("len train set: ", len(train_set))
print("train_set: ", train_set)
ego, neighbors, map_lanes, map_crosswalks, route_lanes, ego_future_gt, neighbors_future_gt, first_stage, second_stage = train_set[0]


len train set:  1
train_set:  <train_utils.DrivingData object at 0x7f5dfdfbdd60>


In [4]:
print("len ego: ", len(ego))
print("len neighbors: ", len(neighbors))
print("len map_lanes: ", len(map_lanes))
print("len map_crosswalks: ", len(map_crosswalks))
print("len route_lanes: ", len(route_lanes))
print("len ego_future_gt: ", len(ego_future_gt))
print("len route_lanes: ", len(route_lanes))
print("len ego_future_gt: ", len(ego_future_gt))
print("len neighbors_future_gt: ", len(neighbors_future_gt))
print("len first_stage: ", len(first_stage))
print("len second_stage: ", len(second_stage))



len ego:  21
len neighbors:  20
len map_lanes:  40
len map_crosswalks:  5
len route_lanes:  10
len ego_future_gt:  80
len route_lanes:  10
len ego_future_gt:  80
len neighbors_future_gt:  10
len first_stage:  10
len second_stage:  10


In [5]:
neighbor = neighbors[0]
print("neighbor: ", neighbor)

neighbor:  [[-1.69942074e+01 -9.70942751e-02 -3.26274359e-03  1.32518077e+00
  -1.40478015e-02  4.15088376e-04  4.37049246e+00  1.82270801e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.67705040e+01  1.45155443e-02 -2.84765521e-03  1.60856521e+00
  -1.47808194e-02 -5.75511949e-03  4.37055159e+00  1.83040953e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.64909954e+01  1.42736183e-02 -4.41359542e-03  1.75110054e+00
  -8.00150633e-03 -5.79216983e-03  4.37110329e+00  1.84281862e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.64630928e+01 -4.16522510e-02 -4.00591595e-03  1.93580914e+00
  -6.53553009e-03  5.61464229e-04  4.39109659e+00  1.84603357e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.61835842e+01 -4.18941788e-02 -4.30131936e-03  2.18200779e+00
  -4.81873751e-03  4.60099755e-03  4.42417240e+00  1.84593010e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.59040756e+01 -4.21361029e-02 -3.08585400e-03  2.36826086e+00
  -1.398

In [6]:
data = np.load(path[0])
print("data files: ", data.files)

print("token: ", data['token'])
print(data['ego_agent_past'].shape)

data files:  ['map_name', 'token', 'ego_agent_past', 'ego_agent_future', 'first_stage_ego_trajectory', 'second_stage_ego_trajectory', 'neighbor_agents_past', 'neighbor_agents_future', 'map_lanes', 'map_crosswalks', 'route_lanes']
token:  7fc811bcf45f5e79
(21, 7)


In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [8]:
# test inputs
train_loader = DataLoader(train_set, batch_size= 12, num_workers=os.cpu_count())

#prepare data
with tqdm(train_loader, desc="Training", unit = "batch") as deta_epoch:
  for batch in deta_epoch:
    inputs = {'ego_agent_past': batch[0].to(device),
              'neighbor_agents_past': batch[1].to(device),
              'map_lanes': batch[2].to(device),
              'map_crosswalks': batch[3].to(device),
              'route_lanes': batch[4].to(device)
              }
    ego_gt_future = batch[5].to(device)
    neighbors_gt_future = batch[6].to(device)
    neighbors_future_valid = torch.ne(neighbors_gt_future[..., :3], 0)
    print("inputs: ", inputs)



Training: 100%|██████████| 1/1 [00:06<00:00,  6.75s/batch]

inputs:  {'ego_agent_past': tensor([[[-8.0778e+00, -4.8910e-02,  1.1531e-02,  2.0099e+00, -4.2228e-02,
           1.7718e+00, -9.4090e-02],
         [-7.7983e+00, -4.9152e-02,  1.1245e-02,  2.2141e+00, -5.4906e-02,
           1.8612e+00, -1.4924e-01],
         [-7.5467e+00,  6.5320e-03,  1.1099e-02,  2.4225e+00, -5.6634e-02,
           1.7653e+00,  2.1776e-03],
         [-7.2672e+00,  6.2901e-03,  1.1107e-02,  2.6288e+00, -6.0398e-02,
           1.8552e+00, -1.1647e-01],
         [-7.2114e+00, -1.0556e-01,  1.1141e-02,  2.8362e+00, -6.9311e-02,
           1.9565e+00, -2.0264e-01],
         [-6.7082e+00,  5.8062e-03,  1.1089e-02,  3.0483e+00, -7.7475e-02,
           2.0824e+00,  8.3421e-02],
         [-6.4287e+00,  5.5643e-03,  1.0864e-02,  3.2604e+00, -7.1105e-02,
           2.0798e+00, -1.2008e-01],
         [-6.1213e+00, -5.0604e-02,  1.0791e-02,  3.4628e+00, -8.1433e-02,
           1.9574e+00,  1.5863e-02],
         [-5.8418e+00, -5.0845e-02,  1.0486e-02,  3.6699e+00, -7.1902e-02,
 

In [ ]:
ego = inputs['ego_agent_past']

print("ego shape: ", ego.shape)
#test ego agent encoder
ego_agent_encoder = nn.LSTM(7, 256, 2, batch_first=True).cuda()
traj, _ = ego_agent_encoder(ego)
print(traj.shape)

ego shape:  torch.Size([1, 21, 7])
neighbors shape:  torch.Size([1, 20, 21, 11])
actors shape:  torch.Size([1, 21, 21, 5])
torch.Size([1, 21, 256])


In [21]:
neighbors = inputs['neighbor_agents_past']
print("neighbors shape: ", neighbors.shape)
actors = torch.cat([ego[:, None, :, :5], neighbors[..., :5]], dim=1)
print("actors shape: ", actors.shape)

neighbor_agent_encoder = nn.LSTM(11, 256, 2, batch_first = True).cuda()
print(neighbors[:, 0].shape)
encoded_neighbors = [neighbor_agent_encoder(neighbors[:, i]) for i in range(neighbors.shape[1])]
print("encoded_neighbors: ", encoded_neighbors[0])


neighbors shape:  torch.Size([1, 20, 21, 11])
actors shape:  torch.Size([1, 21, 21, 5])
torch.Size([1, 21, 11])
encoded_neighbors:  (tensor([[[-4.0402e-02, -4.3178e-02, -5.1252e-05,  ...,  7.9031e-03,
          -5.4788e-03,  1.8944e-02],
         [-7.5975e-02, -7.3312e-02,  2.9989e-04,  ...,  2.2091e-04,
          -5.4986e-03,  4.1721e-02],
         [-1.0067e-01, -9.1844e-02,  9.6263e-04,  ..., -1.2873e-02,
          -4.0390e-03,  5.8303e-02],
         ...,
         [-1.0412e-01, -9.2199e-02,  4.6298e-02,  ..., -5.8274e-02,
          -2.1003e-02,  5.9284e-02],
         [-9.6835e-02, -8.8676e-02,  4.8576e-02,  ..., -5.7341e-02,
          -2.3600e-02,  5.7410e-02],
         [-8.8995e-02, -8.4797e-02,  5.0665e-02,  ..., -5.6169e-02,
          -2.6474e-02,  5.5398e-02]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>), (tensor([[[ 1.3870e-01, -8.6138e-02,  6.7852e-02, -2.1498e-02, -7.7290e-02,
           1.3569e-01, -4.0938e-01, -3.2706e-01,  1.8527e-01,  3.3868e-02,
          -2.5291

In [ ]:
#test mask
actors_mask = torch.eq(actors[:,:,-1].sum(-1), 0) #选取时间最近的轨迹点的五个维度的特征，如果均为0，那么该代理数据需要被mask


tensor([[[-4.6566e-10, -2.3283e-10,  7.3614e-09,  6.1296e+00, -1.1917e-01],
         [-1.0034e+01, -4.7217e-02, -7.8354e-03,  5.1880e+00, -3.0893e-02],
         [ 7.5246e+00,  6.7576e+00,  3.6399e-03,  4.9884e+00,  4.7142e-02],
         [-2.8985e+00,  9.7294e+00, -1.1297e-02,  4.3095e+00, -4.6191e-02],
         [-8.7148e+00,  6.8276e+00,  3.7544e-03,  3.8363e+00,  2.0904e-02],
         [ 1.1491e+01,  3.2883e+00, -1.8842e-02,  5.6284e+00, -9.0663e-02],
         [ 8.5056e+00,  9.8313e+00,  3.6096e-02,  5.2889e+00,  1.9472e-01],
         [-1.4135e+01,  9.8509e+00, -5.0950e-04,  3.9425e+00,  3.1334e-03],
         [ 1.1667e+01,  1.3574e+01,  1.0189e-01,  5.7984e+00,  6.4068e-01],
         [ 1.8922e+01, -4.0769e-01, -2.2156e-02,  9.0877e+00, -2.3083e-01],
         [ 2.3565e+01,  3.1660e+00, -1.5898e-02,  6.4382e+00, -4.1499e-02],
         [-2.2914e+01,  6.7280e+00,  5.8258e-03,  3.1815e-01,  2.0855e-02],
         [-2.6777e+01,  4.7039e-01,  2.6013e-02,  1.8029e+00,  2.9922e-02],
         [ 2

In [57]:
#test map lane encoder
from prediction_modules import PositionalEncoding, VectorMapEncoder
_lane_feature = 7
_lane_len = 50
lane_encoder = VectorMapEncoder(_lane_feature, _lane_len).cuda()

map_lanes = inputs['map_lanes']

# print("map_lane.shape: ", map_lanes.shape) #batch, lane nums, sample nums of single lane, feature nums of single point
# print("single lane, first point info: ", map_lanes[0, 0, :, :])

# lane vector map encoder
point_net = nn.Sequential(nn.Linear(_lane_feature, 64), nn.ReLU(), nn.Linear(64, 128), nn.ReLU(), nn.Linear(128, 256)).cuda()
mid_output = point_net(map_lanes)
print("mid_output.shape: ", mid_output.shape)

positional_encoding = PositionalEncoding(max_len= _lane_len)



mid_output.shape:  torch.Size([1, 40, 50, 256])


In [59]:
position = torch.arange(_lane_len).unsqueeze(1)
d_model = 256

print(position.shape)
div_term = torch.exp(torch.arange(0, d_model, 2)) * (- math.log(10000.0) / d_model)
print(torch.arange(0, d_model, 2).shape)
print(div_term.shape)

pe = torch.zeros(_lane_len, 1, d_model).cuda()
print(pe.shape)
print(pe[:, 0, 0::2].shape)

pe[:, 0, 0::2] = torch.sin(position * div_term)
pe[:, 0, 1::2] = torch.cos(position * div_term)

pe = pe.permute(1, 0, 2)

map_lane_encoded = mid_output +pe

torch.Size([50, 1])
torch.Size([128])
torch.Size([128])
torch.Size([50, 1, 256])
torch.Size([50, 128])
